In [ ]:
import flopy
import pyemu
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import herebedragons as hbd
import shutil
from pyemu.emulators import DSI

# Intro

We saw that this awfully long running model (~5min) is going to be a major pain to history match because it takes too long and we dont have a huge (and free) cluster at our disposal...so what can we do? Surrogate models to the rescue!

We wont go into the details on data space inversion (DSI). See the various GMDSI videos and tutorial notebooks for more technical details and background. Here we focus on showing how to implement it with `pyemu`.

What we need:
 - results from a prior monte carlo
 - some data to assimilate
 - some prediction we care about

## New things in this tutorial
 - We introduce `pestpp` new (as of Nov 2025) external run manager. This allows the user to handle paralel forward runs without relying on pest "workers".

# Getting ready
## Load the Prior MC results

In [ ]:
# specify the temporary working folder
org_md = os.path.join('master_prior_mc')
pst = pyemu.Pst(os.path.join(org_md,'pest.pst'))
oe = pst.ies.obsen.copy()
oe.head()

In [ ]:
sim = flopy.mf6.MFSimulation.load(sim_ws='model', load_only=[],verbosity_level=0)
gwf = sim.get_model('gwf')

## Choose the truth

Lets choose one of the realizations as the rtuth, then remove if form the training data. We are going to select an inconvenient truth: one in which the max temperature plume extends a bit farther than most.

In [ ]:
obs = pst.observation_data
obsnmes = obs.loc[obs.oname=="temp"].obsnme.tolist()

# find columns in data[obsnmes] where 50% of the values are below 16.01
cols = oe[obsnmes].columns[(oe[obsnmes] <= 16.005).mean() > 0.9]
print(len(cols), len(obsnmes))

# find col in cols with highest std
stds = oe[cols].std()
cols = stds.sort_values(ascending=False).index.tolist()


target_col = cols[0]
print(target_col)

fig,ax = plt.subplots(1,1,figsize=(4,4))

oe.loc[:,target_col].hist(ax=ax)

truth_index = oe[target_col].sort_values().index.values[-7]
ax.axvline(oe.loc[truth_index, target_col], color='r')
fig.tight_layout();

Lets keep that for later.

In [ ]:
truth = oe.loc[truth_index]

Drop the truth form the training data set

In [ ]:
data = oe.loc[~oe.index.isin([truth_index]), :]
assert data.shape[0] == oe.shape[0] - 1

## Set observation values and weights

Use the simulated values from the truth real as calibration targets.

In [ ]:
obs = pst.observation_data
obs.loc[truth.index,'obsval'] = truth.values
obs.oname.unique()

In [ ]:
obs.loc[target_col]

We have a little utility function in `herebdragons.py` to get the cell ids that correspond to "measured" heads (btw, these match obs locations in the original non-python tutorials):

In [ ]:
calib_obs = hbd.get_obs_cellids(org_md)
calib_obs.head()

Set non-zero weights to these observations. We are history matching for the historical (past) flow stress period. 

We also have measured heads at specified locations, as well as a section of the river that is gauged. Lets just use an arbitrary assumption of stdv of 0.1 and 0.0001 for heads and riv rate, respectively.

In [ ]:
_obs = obs.loc[obs.oname=='heads0'].copy()
_obs['i'] = _obs['i'].astype(int)
_obs.sort_values(by=['i'],inplace=True)

nzobsnmes = _obs.loc[_obs['i'].isin(calib_obs.icpl.values)].obsnme.tolist()
assert len(nzobsnmes) >0

obs.loc[nzobsnmes,'weight'] = 1.0 / 0.1
obs.loc[nzobsnmes,'standard_deviation'] = 0.1

obs.loc[obs.oname=='riv', 'weight'] = 1.0 / 0.0001
obs.loc[obs.oname=='riv', 'standard_deviation'] = 0.0001

Just for fun, lets look at the true K and hyperparameter fields:

In [ ]:
onames =['k', 'npfkpp-aniso', 'npfkpp-bearing', 'npfkpp-corrlen']

fig,axs = plt.subplots(1,4,figsize=(16,4))

for e,oname in enumerate(onames):
    ax = axs[e]
    ax.set_aspect("equal")
    pm = flopy.plot.PlotMapView(model=gwf, ax=ax)

    _obs = obs.loc[obs.oname==oname].copy()
    _obs["i"] = _obs["i"].astype(int)
    _obs.sort_values("i", inplace=True)
    obsnmes = _obs.obsnme.tolist()
    arr = obs.loc[obsnmes].obsval.values
    if oname=='k':
        arr = np.log10(arr)
        oname = "log10(k)"

    pa = pm.plot_array(arr)
    plt.colorbar(pa, ax=ax, shrink=0.5)

    ax.set_title(oname)
    ax.set_xticks([])
    ax.set_yticks([])


fig.tight_layout();
plt.show()
plt.close();

And the true max temperature field. This is our prediction of interest.

In [ ]:
fig,ax = plt.subplots(1,1,figsize=(5,5))

oname = "temp"

ax.set_aspect("equal")
pm = flopy.plot.PlotMapView(model=gwf, ax=ax)

_obs = obs.loc[obs.oname==oname].copy()
_obs["i"] = _obs["i"].astype(int)
_obs.sort_values("i", inplace=True)
obsnmes = _obs.obsnme.tolist()
arr = obs.loc[obsnmes].obsval.values
#f oname=='k':
#    arr = np.log10(arr)

pa = pm.plot_array(arr)
plt.colorbar(pa, ax=ax, shrink=0.5)

ax.set_title(oname)
ax.set_xticks([])
ax.set_yticks([])


fig.tight_layout();
plt.show()
plt.close();

# Start DSI

For DSI, we dont need all the observations we have been tracking. We only really need the non-zero weighted obs and the predictions of interest. 

In [ ]:
obs = pst.observation_data
obs.oname.unique()

keep_obs = obs.loc[obs.weight > 0].obsnme.tolist()
keep_obs.extend(obs.loc[obs.oname=='temp'].obsnme.tolist())

Lets use normal score transformation (usualy a good idea...)

In [ ]:
transforms = [
            {"type":"normal_score"}
            ]

And now we are good to build the DSI surrogate:

In [ ]:
dsi = DSI(pst=pst, #optional...
          data = data[keep_obs],
          transforms=transforms,
           energy_threshold=0.9999, # the truncated-svd energy threshold
          verbose=True)

dsi.fit();

Nothing new so far. But now we can introduce some new functionality in `pyemu.DSI`.

Recall that the dsi "model" is really just a matrix multiplication. We are passing in the `pval` array of parameter values and mulplitying it by the projection matrix. This is easly vectorizable...

Instread of passing each realization of `pvals` once at a time, we can do the same matrix multiplication on an ensemble of `pvals` - all in one go. Traditionaly, `pest` and `pestpp` were not designed for this. However, `pestpp` now allows a user to handle the "paralelisation" externaly. 

Lets quickly go through what vectorization of `DSI` looks like, then demo how to run the external manager with `pestpp-ies`.

First, genera te a dummy ensemble of DSi parameter values:

In [ ]:
pvals = np.random.normal(0,1, size=(5,dsi.s.shape[0]))
pvals.shape

pvals = pd.DataFrame(pvals, index=[f"real_{i}" for i in range(pvals.shape[0])])
pvals.iloc[-1,:] = 0.0
pvals.index.values[-1] = 'base'
pvals

Now we can simply pass that to the `dsi` object and call `.predict()`:

In [ ]:
dsi.predict(pvals)

Or if you wish to do one at a time:

In [ ]:
dsi.predict(pvals.loc['base'])

Eazyy az.

# Setup for pestpp

Right, lets go and setup the DSI pest dir...

key detail now, spceify the `use_runstor=True` argument to setup the forward run using the external manager:

In [ ]:
dsi_t_d = "pst_template_dsi"

dpst = dsi.prepare_pestpp(t_d = dsi_t_d,
                          use_runstor=True)
dpst

We can take a look at what the forward run function looks like:

In [ ]:
#open forward_run.py and print
with open(os.path.join(dsi_t_d, 'forward_run.py'),'r') as f:
    print(f.read())

Get the executables again...

In [ ]:
hbd.get_bins(dsi_t_d)

And we are good to go...sheesh that was hard.
Lets just run for 2 iterations. Why? Because DSI will usualy get a good fit pretty fast. Feel free to experiment with more or less if you like.

Let's go a bit higher with the number of DSi reals that we had for the training data set:

In [ ]:
nreals = 1000

In [ ]:
dpst.control_data.noptmax = 2
dpst.pestpp_options["ies_num_reals"] = nreals

In [ ]:
dpst.write(os.path.join(dsi_t_d, "dsi.pst"),version=2)

# Run pestpp with /e

Right on! We are ready to get cracking. Let's run pestpp-ies and see what we get. 

Make a copy for safekeeping...

In [ ]:
md = f"master_dsi"

if os.path.exists(md):
    shutil.rmtree(md)
shutil.copytree(dsi_t_d, md)

Now we will run `pestpp-ies` with the external run manager option. We do this by calling `pestpp-ies [controlfile].pst /e`. The `/e` trigegrs the external run manager option.

Note that this run is in "serial" as far as `pestpp` is concerned. We are handling the paralelization of the dsi forward runs.

Here we go!

In [ ]:
pyemu.os_utils.run('pestpp-ies dsi.pst /e', cwd=md,verbose=True)

Boom! That should take <1min (on a MacBook it takes ~30sec). Whats even more awesome, is that it scales nicely. You can increase the number of realizations without incurring an linear increase in run time. Give it a go..

# Read pest results

In [ ]:
pst = pyemu.Pst(os.path.join(md,"dsi.pst"))

In [ ]:
pst.ies.phiactual

In [ ]:
obs = pst.observation_data
obs.oname.unique()

Get the posterior observation ensemble

In [ ]:
obsen = pst.ies.obsen.copy()
oe = obsen.loc[2]
oe.head()

See how well we fit the head obs data:

In [ ]:
nzobsnmes = pst.nnz_obs_names

nzobsnmes = pst.nnz_obs_names[:-1]
fig,axs = plt.subplots(1,2,figsize=(6,3))

ax = axs[0]
ax.set_aspect('equal')
ax.set_title('head obs')

[ax.scatter(obs.loc[nzobsnmes].obsval, oe.loc[i,nzobsnmes],c='b',marker='.') for i in oe.index];

xmax = max(ax.get_xlim()[0],ax.get_ylim()[0])
ymax = max(ax.get_xlim()[1],ax.get_ylim()[1])
limax = max(xmax,ymax)
xmin = min(ax.get_xlim()[0],ax.get_ylim()[0])
ymin = min(ax.get_xlim()[1],ax.get_ylim()[1])
limn = min(xmin,ymin)
ax.plot([limn,limax],[limn,limax],'r--')
ax.set_xlim(limn,limax)
ax.set_ylim(limn,limax)


nzobsnmes = pst.nnz_obs_names[-1:]
ax = axs[1]
ax.set_aspect('equal')
ax.set_title(nzobsnmes[0])

[ax.scatter(obs.loc[nzobsnmes].obsval, oe.loc[i,nzobsnmes],c='b',marker='.') for i in oe.index];

xmax = max(ax.get_xlim()[0],ax.get_ylim()[0])
ymax = max(ax.get_xlim()[1],ax.get_ylim()[1])
limax = max(xmax,ymax)
xmin = min(ax.get_xlim()[0],ax.get_ylim()[0])
ymin = min(ax.get_xlim()[1],ax.get_ylim()[1])
limn = min(xmin,ymin)
ax.plot([limn,limax],[limn,limax],'r--')
ax.set_xlim(limn,limax)
ax.set_ylim(limn,limax)

fig.tight_layout();

Make some plots of max temperature:

In [ ]:
oname = "temp"

for i in oe.index.values[-5:]:
    fig,ax = plt.subplots(1,1,figsize=(6,6))


    ax.set_aspect("equal")
    pm = flopy.plot.PlotMapView(model=gwf, ax=ax)
    #pm.plot_grid(alpha=0.1,lw=0.1)

    _obs = obs.loc[obs.oname==oname].copy()
    _obs["i"] = _obs["i"].astype(int)
    _obs.sort_values("i", inplace=True)
    obsnmes = _obs.obsnme.tolist()
    diverg = max(oe.loc[:,obsnmes].values.max(), abs(oe.loc[:,obsnmes].values.min()))
    vmax = np.ceil(diverg)
    vmin = 16 
    
    arr = oe.loc[i,obsnmes].values
    #arr = arr.reshape(61,gwf.modelgrid.ncpl).max(axis=0)
    #f oname=='k':
    #    arr = np.log10(arr)

    pa = pm.plot_array(arr, cmap="Reds", vmin=vmin,vmax=vmax)
    plt.colorbar(pa, ax=ax, shrink=0.5)

    ax.set_title(oname)
    ax.set_xticks([])
    ax.set_yticks([])

    fig.tight_layout();
    plt.show()
    plt.close();

Now lets compare to the truth and look at how data assimilation reduced predictive uncertainty:

In [ ]:

_obs = obs.loc[obs.oname==oname].copy()
_obs["i"] = _obs["i"].astype(int)
_obs.sort_values("i", inplace=True)
obsnmes = _obs.obsnme.tolist()


fig,axs = plt.subplots(2,2,figsize=(12,12))


ax = axs[0,0]
ax.set_aspect("equal")
pm = flopy.plot.PlotMapView(model=gwf, ax=ax)
#pm.plot_grid(alpha=0.1,lw=0.1)

arr = oe.loc[:,obsnmes].mean()


vmax = max(oe.loc[:,obsnmes].values.max(), abs(oe.loc[:,obsnmes].values.min()))
vmin = oe.loc[:,obsnmes].values.min()
vmax = np.ceil(vmax)
vmin = 16 
pa = pm.plot_array(arr, cmap="Reds", vmin=vmin, vmax=vmax)
plt.colorbar(pa, ax=ax, shrink=0.5)

ax.set_title('dsi(mean)')


ax = axs[0,1]
ax.set_aspect("equal")
pm = flopy.plot.PlotMapView(model=gwf, ax=ax)
#pm.plot_grid(alpha=0.1,lw=0.1)

arr = oe.loc[:,obsnmes].std()


vmax = max(oe.loc[:,obsnmes].values.max(), abs(oe.loc[:,obsnmes].values.min()))
vmin = oe.loc[:,obsnmes].values.min()
vmax = np.ceil(vmax)
vmin = 16 
pa = pm.plot_array(arr, cmap="Reds",)# vmin=vmin, vmax=vmax)
plt.colorbar(pa, ax=ax, shrink=0.5)


cell = int(obs.loc[target_col].i)
arr = arr.values
arr[:] = np.nan
arr[cell] = 1.0
pa = pm.plot_array(arr,)# vmin=vmin, vmax=vmax)

ax.set_title('dsi(std)')


ax = axs[1,0]
ax.set_aspect("equal")
pm = flopy.plot.PlotMapView(model=gwf, ax=ax)
#pm.plot_grid(alpha=0.1,lw=0.1)

arr = oe.loc[:,obsnmes].mean()
arr = arr - obs.loc[obsnmes,"obsval"].values

vmax = max(arr.max(), abs(arr.min()))

pa = pm.plot_array(arr, cmap="RdBu", vmin=-vmax,vmax=vmax)
plt.colorbar(pa, ax=ax, shrink=0.5)

cell = int(obs.loc[target_col].i)
arr = arr.values
arr[:] = np.nan
arr[cell] = 1.0
pa = pm.plot_array(arr,)# vmin=vmin, vmax=vmax)

ax.set_title('dsi(mean) - truth')


ax = axs[1,1]
ax.set_aspect("equal")
pm = flopy.plot.PlotMapView(model=gwf, ax=ax)

arr =  (data.loc[:,obsnmes].std() - oe.loc[:,obsnmes].std())/data.loc[:,obsnmes].std()
arr[abs(data.loc[:,obsnmes].std())<1e-3] = 0

pa = pm.plot_array(arr, cmap="RdBu_r",vmax=1,vmin=-1)#,vmin=-vmax,vmax=vmax )
plt.colorbar(pa, ax=ax, shrink=0.5)

ax.set_title('rel std unc reduction [(prior-post)/(prior)]')

for ax in axs.flatten():
    ax.set_xticks([])
    ax.set_yticks([])

fig.tight_layout();
plt.show()
plt.close();

In [ ]:
obs = pst.observation_data
obsnmes = obs.loc[obs.oname=="temp"].obsnme.tolist()

fig,ax = plt.subplots(1,1,figsize=(5,5))

data.loc[:,target_col].hist(ax=ax, alpha=0.5,color='fuchsia')

obsen.loc[0].loc[:,target_col].hist(ax=ax,color='0.5')
oe.loc[:,target_col].hist(ax=ax,alpha=0.5,color='b')

truth_index = oe[target_col].sort_values().index.values[-3]
ax.axvline(obs.loc[target_col].obsval, color='r')
fig.tight_layout();